# Classical Solver for QUBO Optimization

This project uses a **classical integer programming solver** to find the optimal RNA secondary structure by minimizing the QUBO (Quadratic Unconstrained Binary Optimization) Hamiltonian. The QUBO is constructed from the potential stems, pseudoknot penalties, and overlap penalties, and encodes the energy landscape of possible RNA foldings.

We use the **Google OR-Tools CP-SAT solver** to solve this problem. The solver treats each potential stem as a Boolean variable (0 = not selected, 1 = selected) and finds the combination of stems that minimizes the total energy, while automatically respecting the penalties for overlaps and pseudoknots.

**Key steps:**
1. **Preprocessing:** Extract all potential stems, pseudoknots, and overlaps from the RNA sequence.
2. **Model Construction:** Build the QUBO Hamiltonian (linear and quadratic terms) using the preprocessing results.
3. **Optimization:** Use the CP-SAT solver to find the optimal set of stems (the predicted structure).
4. **Evaluation:** Compare the predicted structure to the actual (experimentally determined) structure.

In [2]:
from preprocess_sequence import (
    actual_stems, potential_stems, potential_pseudoknots, potential_overlaps, model, energy
)
from classical_optimizer import solve_qubo_with_cpsat

# Define file paths
subdirectory = "./data"
ct_file = "bpRNA_RFAM_23352.ct.txt"
fasta_file = "bpRNA_RFAM_23352.fasta.txt"

# params
pseudoknot_penalty = 2.0


# 1. Get actual stems (ground truth)
actual = actual_stems(ct_file, fasta_file, subdirectory)
print("Actual stems (ground truth):", actual)

# 2. Get potential stems and related info
pot_stems, mu, rna, seq_len = potential_stems(fasta_file, subdirectory)

# 3. Get potential pseudoknots and overlaps
pot_pks = potential_pseudoknots(pot_stems, pseudoknot_penalty)
pot_ovs = potential_overlaps(pot_stems)

# 4. Build the QUBO model (linear and quadratic terms)
L, Q = model(pot_stems, pot_pks, pot_ovs, mu)

# 5. Solve the QUBO using the classical solver
solution, energy_pred = solve_qubo_with_cpsat(L, Q)

# 6. Extract the predicted stems from the solution
print("Actual stems:", actual)
predicted_stems = [pot_stems[i] for i, v in solution.items() if v == 1]
print("Predicted stems:", predicted_stems)


# 7. Calculate and print the energy of the actual structure
energy_actual = energy(actual, pseudoknot_penalty)
print("Energy of actual structure:", energy_actual)

# 8. Calculate and print the energy of the predicted structure
energy_predicted = energy(predicted_stems, pseudoknot_penalty)
print("Energy of predicted structure:", energy_predicted)

Actual stems (ground truth): [[2, 19, 8], [11, 33, 11], [16, 27, 3]]
[DEBUG] Number of variables: 74
[DEBUG] Number of quadratic terms: 2701
[DEBUG] Sample linear terms: [(0, 102589), (1, 13289), (2, -104310), (3, -16110), (4, 42690)]
[DEBUG] Sample quadratic terms: [((0, 1), 999944000), ((0, 2), 999912000), ((0, 3), 999872000), ((0, 4), 999904000), ((0, 5), 999872000)]
[DEBUG] Starting solver...
[DEBUG] Solver finished with a solution.
Best solution: {0: 0, 1: 0, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0, 41: 0, 42: 0, 43: 0, 44: 0, 45: 0, 46: 0, 47: 0, 48: 0, 49: 0, 50: 0, 51: 0, 52: 0, 53: 0, 54: 0, 55: 0, 56: 0, 57: 0, 58: 0, 59: 0, 60: 0, 61: 0, 62: 0, 63: 0, 64: 0, 65: 0, 66: 0, 67: 0, 68: 0, 69: 0, 70: 0, 71: 0, 72: 0, 73: 1}
Best energy: -245.021
Actual stems:

In [18]:
# Import quantum optimizer
from quantum_optimizer import build_quadratic_program, solve_with_vqe, analyze_result
import numpy as np

# Scale down the problem for quantum solver
def scale_qubo(L, Q, scale_factor=1e-6):
    """Scale down the QUBO coefficients to avoid numerical issues."""
    L_scaled = {k: v * scale_factor for k, v in L.items()}
    Q_scaled = {k: v * scale_factor for k, v in Q.items()}
    return L_scaled, Q_scaled

# Select a subset of variables for quantum solver (due to qubit limitations)
def select_subset(L, Q, n_vars=8):
    """Select a subset of variables for quantum solver based on their position in the RNA sequence.
    
    Args:
        L (dict): Linear terms with variable indices as keys
        Q (dict): Quadratic terms with tuples of variable indices as keys
        n_vars (int): Number of variables to select (takes first n_vars stems)
    
    Returns:
        tuple: (L_subset, Q_subset) containing only terms where both variables
               are within the first n_vars stems
    """
    # Get the first n_vars indices from L
    selected_indices = set(list(L.keys())[:n_vars])
    
    # Filter L to only include selected indices
    L_subset = {k: v for k, v in L.items() if k in selected_indices}
    
    # Filter Q to only include terms where both variables are in selected indices
    Q_subset = {k: v for k, v in Q.items() 
                if k[0] in selected_indices and k[1] in selected_indices}
    
    return L_subset, Q_subset

# Scale and subset the problem
L_scaled, Q_scaled = scale_qubo(L, Q)
L_subset, Q_subset = select_subset(L_scaled, Q_scaled, n_vars=8)

L_subset = {int(k): v for k, v in L_subset.items()}
Q_subset = {(int(i), int(j)): v for (i, j), v in Q_subset.items()}

print("Original problem size:", len(L), "variables")
print("Subset problem size:", len(L_subset), "variables")
print("Selected indices:", sorted(L_subset.keys()))

# -----------------------------------------------------------
# build the sub-problem and solve it with VQE
# -----------------------------------------------------------
qp = build_quadratic_program(L_subset, Q_subset)
print("\nQUBO as QuadraticProgram:")
print(qp.export_as_lp_string())

print("\n--- Quantum VQE Solution ---")
result = solve_with_vqe(qp, max_evals=300)
analyze_result(result)

# -----------------------------------------------------------
# convert VQE bitstring back to original indices
# -----------------------------------------------------------
var_names = [var.name for var in qp.variables]   # keep declaration order

def idx(name: str) -> int:                       # new robust parser
    import re
    m = re.search(r'\d+', name)
    if m is None:
        raise ValueError(f"No digits in variable name {name!r}")
    return int(m.group())                        # already 0-based

quantum_solution = {idx(n): val
                    for n, val in zip(var_names, result.x)}

# classical CPSAT solution restricted to the same indices
subset_solution = {k: solution[k] for k in quantum_solution}

# -----------------------------------------------------------
# energy comparison  (no –1 anywhere)
# -----------------------------------------------------------
def val_L(i):
    return L_subset.get(i, L_subset.get(str(i), 0.0))

def val_Q(i, j):
    return Q_subset.get((i, j), Q_subset.get((str(i), str(j)), 0.0))

subset_energy = (
    sum(val_L(i) * subset_solution[i] for i in subset_solution) +
    sum(val_Q(i, j) * subset_solution[i] * subset_solution[j]
        for (i, j) in Q_subset)          # keys already as-stored
)

print("\n--- Comparison ---")
print("Classical solution for subset:", subset_solution)
print("Classical energy for subset :", subset_energy)
print("Quantum   solution          :", quantum_solution)
print("Quantum   energy            :", result.fval)

print("\n--- Stem Comparison ---")
print("Classical stems:",
      [pot_stems[i] for i, v in subset_solution.items() if v == 1])
print("Quantum stems  :",
      [pot_stems[i] for i, v in quantum_solution.items() if v == 1])

Original problem size: 74 variables
Subset problem size: 8 variables
Selected indices: [0, 1, 2, 3, 4, 5, 6, 7]

QUBO as QuadraticProgram:
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: 0.000102590000 x_0 + 0.000013290000 x_1 - 0.000104310000 x_2
      - 0.000016110000 x_3 + 0.000042690000 x_4 - 0.000016110000 x_5
      + 0.000072090000 x_6 - 0.000016110000 x_7 + [ 1.999888000000 x_0*x_1
      + 1.999824000000 x_0*x_2 + 1.999744000000 x_0*x_3 + 1.999808000000 x_0*x_4
      + 1.999744000000 x_0*x_5 + 1.999840000000 x_0*x_6 + 1.999744000000 x_0*x_7
      + 1.999692000000 x_1*x_2 + 1.999776000000 x_1*x_3 + 1.999664000000 x_1*x_4
      + 1.999552000000 x_1*x_5 + 1.999720000000 x_1*x_6 + 1.999552000000 x_1*x_7
      + 1.999648000000 x_2*x_3 + 1.999472000000 x_2*x_4 + 1.999296000000 x_2*x_5
      + 1.999560000000 x_2*x_6 + 1.999296000000 x_2*x_7 + 1.999616000000 x_3*x_4
      + 1.999488000000 x_3*x_5 + 1.999680000000 x_3*x_6 + 1.999488000

### Putting it all together

In [1]:
# Import required modules
from preprocess_sequence import (
    actual_stems, potential_stems, potential_pseudoknots, potential_overlaps, model, energy
)
from classical_optimizer import solve_qubo_with_cpsat
from quantum_optimizer import build_quadratic_program, solve_with_vqe, analyze_result
import numpy as np

# Define file paths and parameters
subdirectory = "./data"
ct_file = "bpRNA_RFAM_23352.ct.txt"
fasta_file = "bpRNA_RFAM_23352.fasta.txt"
pseudoknot_penalty = 2.0
n_vars = 12  # number of variables to select

# Helper functions
def scale_qubo(L, Q, scale_factor=1):
    """Scale down the QUBO coefficients to avoid numerical issues."""
    L_scaled = {k: v * scale_factor for k, v in L.items()}
    Q_scaled = {k: v * scale_factor for k, v in Q.items()}
    return L_scaled, Q_scaled

def select_subset(L, Q, n_vars=8):
    """Select a subset of variables for quantum solver based on their position in the RNA sequence."""
    # Get the first n_vars indices from L
    selected_indices = set(list(L.keys())[:n_vars])
    
    # Filter L and Q to only include selected indices
    L_subset = {k: v for k, v in L.items() if k in selected_indices}
    Q_subset = {k: v for k, v in Q.items() 
                if k[0] in selected_indices and k[1] in selected_indices}
    
    return L_subset, Q_subset

# 1. Get potential stems and build the QUBO model
pot_stems, mu, rna, seq_len = potential_stems(fasta_file, subdirectory)
pot_pks = potential_pseudoknots(pot_stems, pseudoknot_penalty)
pot_ovs = potential_overlaps(pot_stems)
L, Q = model(pot_stems, pot_pks, pot_ovs, mu)

# 2. Scale and subset the problem
L_scaled, Q_scaled = scale_qubo(L, Q)
L_subset, Q_subset = select_subset(L_scaled, Q_scaled, n_vars=n_vars)

# Convert keys to integers for consistency
L_subset = {int(k): v for k, v in L_subset.items()}
Q_subset = {(int(i), int(j)): v for (i, j), v in Q_subset.items()}

print("\n--- Debug: Linear Terms (L_subset) ---")
for idx, val in sorted(L_subset.items()):
    print(f"L[{idx}] = {val}")

print("\n--- Debug: Quadratic Terms (Q_subset) ---")
for (i, j), val in sorted(Q_subset.items()):
    print(f"Q[({i}, {j})] = {val}")


print("Problem size:", len(L_subset), "variables")
print("Selected indices:", sorted(L_subset.keys()))

# 3. Solve with classical solver (CPSAT)
print("\n--- Classical CPSAT Solution ---")
classical_solution, classical_energy = solve_qubo_with_cpsat(L_subset, Q_subset)
print("Classical solution:", classical_solution)
print("Classical energy  :", classical_energy)

# 4. Solve with quantum solver (VQE)
print("\n--- Quantum VQE Solution ---")
qp = build_quadratic_program(L_subset, Q_subset)
result = solve_with_vqe(qp, max_evals=1000)
analyze_result(result)

# Convert quantum solution to dictionary
var_names = [var.name for var in qp.variables]
quantum_solution = {int(name[1:]): val for name, val in zip(var_names, result.x)}
print("Quantum solution:", quantum_solution)
print("Quantum energy  :", result.fval)

# 5. Compare solutions
print("\n--- Solution Comparison ---")
print("Energy difference (Quantum - Classical):", result.fval - classical_energy)

print("\n--- Stem Comparison ---")
print("Classical stems:", [pot_stems[i] for i, v in classical_solution.items() if v == 1])
print("Quantum stems  :", [pot_stems[i] for i, v in quantum_solution.items() if v == 1])


--- Debug: Linear Terms (L_subset) ---
L[0] = 102.58999999999997
L[1] = 13.289999999999992
L[2] = -104.31
L[3] = -16.110000000000014
L[4] = 42.69
L[5] = -16.110000000000014
L[6] = 72.08999999999997
L[7] = -16.110000000000014
L[8] = -74.91000000000003
L[9] = -133.70999999999998
L[10] = -192.51
L[11] = 42.69

--- Debug: Quadratic Terms (Q_subset) ---
Q[(0, 1)] = 944
Q[(0, 2)] = 912
Q[(0, 3)] = 872.0
Q[(0, 4)] = 904.0
Q[(0, 5)] = 872.0
Q[(0, 6)] = 920.0
Q[(0, 7)] = 872.0
Q[(0, 8)] = 840.0
Q[(0, 9)] = 808.0
Q[(0, 10)] = 776.0
Q[(0, 11)] = 904.0
Q[(1, 2)] = 846
Q[(1, 3)] = 888
Q[(1, 4)] = 832.0
Q[(1, 5)] = 776.0
Q[(1, 6)] = 860.0
Q[(1, 7)] = 776.0
Q[(1, 8)] = 720.0
Q[(1, 9)] = 664.0
Q[(1, 10)] = 608.0
Q[(1, 11)] = 832.0
Q[(2, 3)] = 824
Q[(2, 4)] = 736.0
Q[(2, 5)] = 648.0
Q[(2, 6)] = 780.0
Q[(2, 7)] = 648.0
Q[(2, 8)] = 560.0
Q[(2, 9)] = 472.0
Q[(2, 10)] = 384.0
Q[(2, 11)] = 736.0
Q[(3, 4)] = 808.0
Q[(3, 5)] = 744.0
Q[(3, 6)] = 840.0
Q[(3, 7)] = 744.0
Q[(3, 8)] = 680.0
Q[(3, 9)] = 616.0
Q[(3